In [38]:
# Load Data
import pandas as pd
# data = pd.read_csv("https://raw.githubusercontent.com/lavibula/ML20222.PredictionBitcoin/main/data/MinMax_Normalized_data.csv")
data = pd.read_csv(r"C:\Users\Administrator\OneDrive - Hanoi University of Science and Technology\ITE10 - Data Science and AI - HUST\20222\ML\Source_Codes\ML20222.PredictionBitcoin\data\MinMax_Normalized_data.csv")
data = data.drop(['Unnamed: 0'], axis=1)
# print(data.info())


In [39]:
# Thêm data từ wiki with 3 features: edit_count,	sentiment,	neg_sentiment

wiki = pd.read_csv("wiki_dataquestio_edited.csv")
# print(wiki.info())
data = pd.merge(data, wiki, on='Date', how='inner') #quan sát info 2 df, 
                       #how=inner: ghép chéo-lấy các hàng chung trong cả 2 df #outer
data.set_index('Date', inplace=True) #inplace=True; thay đổi được áp dụng trực tiếp vào df gốc
# print(data.info())

In [40]:
# Add col: BTC_target: 1/0 so với hôm sau, hàng cuối có cột này =0
data["BTC_tomorrow"] = data["BTC_close"].shift(-1)
data["BTC_target"] = (data["BTC_tomorrow"] > data["BTC_close"]).astype(int)
data["BTC_target"].value_counts()  # BTC_target: BTC_direction, compare BTC_close with BTC_tomorrow.

data = data.drop("BTC_tomorrow", axis=1)
data
data.to_csv('data_compare_tomorrow')

In [41]:
# Improving precision with trends: find  the trend in a few different col
def compute_rolling(btc):
    horizons = [2,7,60,365]
    new_predictors = ["BTC_close", "sentiment", "neg_sentiment"]

    for horizon in horizons:
        rolling_averages = btc.rolling(horizon, min_periods=1).mean()

        ratio_column = f"close_ratio_{horizon}"
        btc[ratio_column] = btc["BTC_close"] / rolling_averages["BTC_close"]
        
        edit_column = f"edit_{horizon}"
        btc[edit_column] = rolling_averages["edit_count"]

        rolling = btc.rolling(horizon, closed='left', min_periods=1).mean()
        trend_column = f"trend_{horizon}"
        btc[trend_column] = rolling["BTC_target"]

        new_predictors+= [ratio_column, trend_column, edit_column]
    return btc, new_predictors

data, new_predictors = compute_rolling(data.copy())
data

,BTC_close,BTC_open,BTC_high,BTC_low,BTC_volume,Active_Addr_Cnt,Difficulty,Mean_Block_Size(in_bytes),Sum_Block_Weight,Total_Fees(USD),...,trend_2,close_ratio_7,edit_7,trend_7,close_ratio_60,edit_60,trend_60,close_ratio_365,edit_365,trend_365
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-20,0.000000,0.000000,0.000000,0.000002,0.000090,0.000387,0.000000,0.000266,0.000667,0.000000e+00,...,NaN,NaN,2.266667,NaN,NaN,2.266667,NaN,NaN,2.266667,NaN
2010-07-21,0.000000,0.000000,0.000000,0.000002,0.000410,0.000275,0.000000,0.000074,0.000285,0.000000e+00,...,0.0,NaN,2.433333,0.000000,NaN,2.433333,0.000000,NaN,2.433333,0.000000
2010-07-22,0.000000,0.000000,0.000000,0.000002,0.001991,0.000136,0.000000,0.000062,0.000157,0.000000e+00,...,0.0,NaN,2.500000,0.000000,NaN,2.500000,0.000000,NaN,2.500000,0.000000
2010-07-23,0.000000,0.000000,0.000000,0.000002,0.002231,0.000181,0.000000,0.000045,0.000169,2.831739e-11,...,0.0,NaN,2.500000,0.000000,NaN,2.500000,0.000000,NaN,2.500000,0.000000
2010-07-24,0.000000,0.000000,0.000000,0.000002,0.000330,0.000403,0.000000,0.000435,0.001140,0.000000e+00,...,0.0,NaN,2.500000,0.000000,NaN,2.500000,0.000000,NaN,2.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-04,0.296171,0.293684,0.290152,0.295391,0.158201,0.566751,0.646868,0.341270,0.511340,9.795753e-03,...,0.5,0.999692,0.342857,0.571429,0.907998,0.270556,0.400000,0.505714,0.891050,0.473973
2022-09-05,0.293109,0.296168,0.290515,0.296235,0.238693,0.689834,0.646868,0.467016,0.626500,1.317211e-02,...,0.5,0.992920,0.357143,0.428571,0.899866,0.267222,0.383333,0.501596,0.890776,0.471233
2022-09-06,0.278201,0.293110,0.292347,0.282254,0.402518,0.678658,0.646868,0.497418,0.614759,1.353347e-02,...,0.0,0.949263,0.376190,0.428571,0.855929,0.264444,0.383333,0.477207,0.890594,0.468493


In [42]:

mean_values = data.mean() # Tính giá trị trung bình của các cột
data = data.fillna(mean_values) # Thay NaN = gt tb các cột
data





,BTC_close,BTC_open,BTC_high,BTC_low,BTC_volume,Active_Addr_Cnt,Difficulty,Mean_Block_Size(in_bytes),Sum_Block_Weight,Total_Fees(USD),...,trend_2,close_ratio_7,edit_7,trend_7,close_ratio_60,edit_60,trend_60,close_ratio_365,edit_365,trend_365
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-20,0.000000,0.000000,0.000000,0.000002,0.000090,0.000387,0.000000,0.000266,0.000667,0.000000e+00,...,0.485337,1.012154,2.266667,0.485031,1.161978,2.266667,0.482657,1.879515,2.266667,0.468371
2010-07-21,0.000000,0.000000,0.000000,0.000002,0.000410,0.000275,0.000000,0.000074,0.000285,0.000000e+00,...,0.000000,1.012154,2.433333,0.000000,1.161978,2.433333,0.000000,1.879515,2.433333,0.000000
2010-07-22,0.000000,0.000000,0.000000,0.000002,0.001991,0.000136,0.000000,0.000062,0.000157,0.000000e+00,...,0.000000,1.012154,2.500000,0.000000,1.161978,2.500000,0.000000,1.879515,2.500000,0.000000
2010-07-23,0.000000,0.000000,0.000000,0.000002,0.002231,0.000181,0.000000,0.000045,0.000169,2.831739e-11,...,0.000000,1.012154,2.500000,0.000000,1.161978,2.500000,0.000000,1.879515,2.500000,0.000000
2010-07-24,0.000000,0.000000,0.000000,0.000002,0.000330,0.000403,0.000000,0.000435,0.001140,0.000000e+00,...,0.000000,1.012154,2.500000,0.000000,1.161978,2.500000,0.000000,1.879515,2.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-04,0.296171,0.293684,0.290152,0.295391,0.158201,0.566751,0.646868,0.341270,0.511340,9.795753e-03,...,0.500000,0.999692,0.342857,0.571429,0.907998,0.270556,0.400000,0.505714,0.891050,0.473973
2022-09-05,0.293109,0.296168,0.290515,0.296235,0.238693,0.689834,0.646868,0.467016,0.626500,1.317211e-02,...,0.500000,0.992920,0.357143,0.428571,0.899866,0.267222,0.383333,0.501596,0.890776,0.471233
2022-09-06,0.278201,0.293110,0.292347,0.282254,0.402518,0.678658,0.646868,0.497418,0.614759,1.353347e-02,...,0.000000,0.949263,0.376190,0.428571,0.855929,0.264444,0.383333,0.477207,0.890594,0.468493
